In [5]:
!pip install pandas

You should consider upgrading via the 'C:\Users\utente\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import os

def leggi_e_salva_file(input_file, output_file, nrows):
    # Leggi il file di input con specifica del delimitatore usando regex
    df = pd.read_csv(input_file, nrows=nrows, sep='\t', engine='python', quoting=3)

    # Salva i primi "nrows" campi nel nuovo file
    df.to_csv(output_file, index=False)



In [3]:
def process_data(interactions_file, tracks_file, artists_file, output_file):
    # Leggi i dati di interazione in un DataFrame pandas
    interactions_data = pd.read_csv(interactions_file, delimiter=',')

    # Rinomina le colonne per rimuovere i caratteri di escape
    interactions_data = interactions_data.rename(columns={
        'user_id:token': 'users_id',
        'tracks_id:token': 'tracks_id',
        'timestamp:float': 'timestamp'
    })

    # Calcola il numero di interazioni per ogni coppia utente-traccia
    user_interaction_counts = interactions_data.groupby(['users_id', 'tracks_id']).size().reset_index().rename(columns={0: 'interaction_count'})

    # Filtra le tracce ascoltate almeno 20 volte da un utente
    user_filtered_data = user_interaction_counts[user_interaction_counts['interaction_count'] >= 20]

    # Leggi i dati delle tracce in un DataFrame pandas
    tracks_data = pd.read_csv(tracks_file, sep='\t', names=['tracks_id', 'track_name', 'artist_id'])

    # Esegui il merge con i dati delle tracce
    merged_data = pd.merge(user_filtered_data, tracks_data, on='tracks_id', how='left')

    # Calcola il numero totale di ascolti per ogni traccia
    track_playcount = interactions_data.groupby('tracks_id')['tracks_id'].count().reset_index(name='total_playcount')

    # Unisci i dati filtrati con i dati totali di playcount per traccia
    final_merged_data = pd.merge(merged_data, track_playcount, on='tracks_id', how='left')

    # Leggi i dati degli artisti in un DataFrame pandas
    artists_data = pd.read_csv(artists_file, sep='\t', names=['artist_id', 'artist_name'])
    
    # Convert 'artist_id' column in artists_data to the same data type as in final_merged_data
    artists_data['artist_id'] = artists_data['artist_id'].astype(final_merged_data['artist_id'].dtype)

    # Esegui il merge con i dati degli artisti
    final_merged_data = pd.merge(final_merged_data, artists_data, on='artist_id', how='left')

    # Filtra le tracce ascoltate almeno 10 volte in totale
    final_filtered_data = final_merged_data[final_merged_data['total_playcount'] >= 10]

    # Seleziona le colonne desiderate
    final_columns = ['users_id', 'tracks_id', 'interaction_count', 'total_playcount', 'track_name', 'artist_name']

    # Salva l'output in un file CSV
    final_filtered_data[final_columns].to_csv(output_file, index=False)



In [7]:
if __name__ == "__main__":
    # Percorso del file delle interazioni utente-traccia
    percorso_file_input = '../LFM-1b/lfm1b-tracks.inter'

    # Chiedi all'utente di inserire il numero di righe da leggere
    inp = input("Inserisci il numero di MILIONI di righe da leggere: ")
    nrows = float(inp) * 1000000

    # Imposta il percorso per il nuovo file
    percorso_file_output = "tracks.csv"

    # Esegui la funzione con il numero specificato di righe
    leggi_e_salva_file(percorso_file_input, percorso_file_output, nrows)
    
    interactions_file = percorso_file_output
    tracks_file = "../LFM-1b/LFM-1b_tracks.txt"
    artists_file = "../LFM-1b/LFM-1b_artists.txt"
    output_file = "sample.csv"

    # Esegui il processo di elaborazione dei dati
    process_data(interactions_file, tracks_file, artists_file, output_file)
    os.remove(percorso_file_output)

Inserisci il numero di MILIONI di righe da leggere:  20
